In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, KFold, GridSearchCV
from sklearn import metrics
from datetime import datetime as dt
import statsmodels.api as sm
import warnings
warnings.filterwarnings("ignore")
from sklearn.feature_selection import RFE
from sklearn.model_selection import cross_val_score

In [2]:
data=pd.read_csv("final30var.csv")

In [5]:
tt=data.loc[data['Date']<='2010-10-31']

In [6]:
oot=data.loc[data['Date']>='2010-11-01']

In [7]:
y_orig=tt.iloc[:,[32]]
X_orig=tt.iloc[:,0:30]

In [8]:
x_oot=oot.iloc[:,0:30]
y_oot=oot.iloc[:,[32]]

In [9]:
X_orig.columns

Index(['Cardnum_max_30', 'Cardnum_mean_0', 'Cardnum_total_1',
       'Cardnum_total_14', 'Cardnum_total_30', 'Merchnum_total_1',
       'amount/amount1Cardnum30', 'card_merch_max_3', 'card_merch_max_7',
       'card_merch_median_3', 'card_merch_total_1', 'card_merch_total_7',
       'card_state_max_1', 'card_state_max_14', 'card_state_max_30',
       'card_state_mean_1', 'card_state_total_14', 'card_state_total_3',
       'card_state_total_30', 'card_zip_max_1', 'card_zip_max_14',
       'card_zip_mean_1', 'card_zip_mean_3', 'card_zip_median_3',
       'card_zip_total_1', 'card_zip_total_14', 'card_zip_total_30',
       'card_zip_total_7', 'Cardnum_max_0', 'card_merch_mean_3'],
      dtype='object')

### Models

## TTL 25variables

In [20]:
niter=0
nitermax=10
FDR3=pd.DataFrame(np.zeros((nitermax,3)),columns=('trn','tst','oot'))
FDR3.head()

,trn,tst,oot
0,0.0,0.0,0.0
1,0.0,0.0,0.0
2,0.0,0.0,0.0
3,0.0,0.0,0.0
4,0.0,0.0,0.0


In [12]:
col_30=['Cardnum_max_30', 'Cardnum_mean_0', 'Cardnum_total_1',
       'Cardnum_total_14', 'Cardnum_total_30', 'Merchnum_total_1',
       'amount/amount1Cardnum30', 'card_merch_max_3', 'card_merch_max_7',
       'card_merch_median_3', 'card_merch_total_1', 'card_merch_total_7',
       'card_state_max_1', 'card_state_max_14', 'card_state_max_30',
       'card_state_mean_1', 'card_state_total_14', 'card_state_total_3',
       'card_state_total_30', 'card_zip_max_1', 'card_zip_max_14',
       'card_zip_mean_1', 'card_zip_mean_3', 'card_zip_median_3',
       'card_zip_total_1', 'card_zip_total_14', 'card_zip_total_30',
       'card_zip_total_7', 'Cardnum_max_0', 'card_merch_mean_3']

In [13]:
model = LogisticRegression()
rfe = RFE(model, 25)
fit = rfe.fit(X_orig, y_orig)
ranking=fit.ranking_.tolist()

In [15]:
col=col_30
ranking=ranking

In [16]:
df = pd.DataFrame(list(zip(col, ranking)), 
               columns =['Name', 'rank']) 
df=df.sort_values(by=['rank'], ascending=True)
df=df.loc[df['rank']==1]

In [17]:
col_25=df['Name'].tolist()

In [18]:
X_25=tt[col_25]
y_25=tt.iloc[:,[32]]
x_oot25 =oot[col_25]

In [21]:
x_oot_orig=x_oot25.copy()
for niter in range(nitermax):
    X_train, X_test, y_train, y_test = train_test_split(X_25, y_25, test_size=0.3)
    model=LogisticRegression(penalty='l2',C=0.01,n_jobs=-1)
    
    X_train_array = X_train.values
    X_test_array = X_test.values
    x_oot_array=x_oot.values
    y_train_array = y_train.values
    y_test_array = y_test.values
    y_oot_array = y_oot.values
    
    x_oot=x_oot_orig.copy()
    x_trn_eval=X_train.copy()
    x_tst_eval=X_test.copy()
    x_oot_eval=x_oot.copy()
    
    model.fit(X_train,y_train.values)
    
    predictions=model.predict_proba(X_train)[:,1]
    
    x_trn_eval['predicted']=predictions
    x_trn_eval['Fraud']=y_train['Fraud']
    toprows=int(round(x_trn_eval.shape[0]*0.03))
    temp=x_trn_eval.sort_values('predicted',ascending=False).head(toprows)
    needed=temp.loc[:,'Fraud']
    FDR3.loc[niter,'trn']=sum(needed)/sum(x_trn_eval.loc[:,'Fraud'])
    
    predictions=model.predict_proba(X_test)[:,1]
    
    x_tst_eval['predicted']=predictions
    x_tst_eval['Fraud']=y_test['Fraud']
    toprows=int(round(x_tst_eval.shape[0]*0.03))
    temp=x_tst_eval.sort_values('predicted',ascending=False).head(toprows)
    needed=temp.loc[:,'Fraud']
    FDR3.loc[niter,'tst']=sum(needed)/sum(x_tst_eval.loc[:,'Fraud'])
    
    predictions=model.predict_proba(x_oot)[:,1]
    
    x_oot_eval['predicted']=predictions
    x_oot_eval['Fraud']=y_oot['Fraud']
    toprows=int(round(x_oot_eval.shape[0]*0.03))
    temp=x_oot_eval.sort_values('predicted',ascending=False).head(toprows)
    needed=temp.loc[:,'Fraud']
    FDR3.loc[niter,'oot']=sum(needed)/sum(x_oot_eval.loc[:,'Fraud'])
    print(FDR3.loc[niter,'trn'],FDR3.loc[niter,'tst'],FDR3.loc[niter,'oot'])

0.6875 0.6583333333333333 0.40782122905027934
0.6699834162520729 0.6931407942238267 0.41899441340782123
0.6704180064308681 0.6976744186046512 0.4134078212290503
0.6909090909090909 0.6545454545454545 0.4022346368715084
0.7037643207855974 0.6319702602230484 0.39664804469273746
0.6881028938906752 0.6666666666666666 0.44692737430167595
0.694078947368421 0.6397058823529411 0.41899441340782123
0.6737012987012987 0.678030303030303 0.4245810055865922
0.6661184210526315 0.7279411764705882 0.4301675977653631
0.6530944625407166 0.7180451127819549 0.4581005586592179


In [22]:
FDR3.to_csv('1.csv')

## GridSearch

In [27]:
logistic=LogisticRegression()
penalty=['l1','l2']
C=np.logspace(0,4,10)
hyperparameters=dict(C=C,penalty=penalty)
clf=GridSearchCV(logistic, hyperparameters,cv=3,verbose=0)
best_model=clf.fit(X_orig,y_orig)

In [28]:
print(best_model.get_params())

{'cv': 3, 'error_score': nan, 'estimator__C': 1.0, 'estimator__class_weight': None, 'estimator__dual': False, 'estimator__fit_intercept': True, 'estimator__intercept_scaling': 1, 'estimator__l1_ratio': None, 'estimator__max_iter': 100, 'estimator__multi_class': 'auto', 'estimator__n_jobs': None, 'estimator__penalty': 'l2', 'estimator__random_state': None, 'estimator__solver': 'lbfgs', 'estimator__tol': 0.0001, 'estimator__verbose': 0, 'estimator__warm_start': False, 'estimator': LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False), 'iid': 'deprecated', 'n_jobs': None, 'param_grid': {'C': array([1.00000000e+00, 2.78255940e+00, 7.74263683e+00, 2.15443469e+01,
       5.99484250e+01, 1.66810054e+02, 4.64158883e+02, 1.2915